In [96]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.agents import tool
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader, YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
import textwrap
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.tools import Tool
import langchain
import json
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain.schema import Document
from typing import Iterator, List, Literal, Optional, Sequence, Union
from langchain.document_loaders.blob_loaders import FileSystemBlobLoader
import os
import openai
from langchain.document_loaders import TextLoader
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

In [18]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [19]:

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

##  Download Video in Save directory
## Transcribe the videos to text
## keep the transcript in document 


# 1 Download Youtube Video

In [20]:

#  Moive Clip
urls = ["https://www.youtube.com/watch?v=5Ay5GqJwHF8&list=PL86SiVwkw_oc8r_X6PL6VqcQ7FTX4923M&index=1"]

# Directory to save audio files
save_dir = "/Users/bpakra200/Downloads/YouTubeclip"

try:
    # Transcribe the videos to text
    loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParser())

    docs = loader.load()
    print("After load")
    if len(docs) == 0:
        print("No documents loaded.")
    else:
        for doc in docs:
            # Process the loaded documents
            print(doc)
except Exception as e:
    print("Error loading documents:", str(e))   

[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=5Ay5GqJwHF8&list=PL86SiVwkw_oc8r_X6PL6VqcQ7FTX4923M&index=1
[youtube:tab] Downloading just the video 5Ay5GqJwHF8 because of --no-playlist
[youtube] Extracting URL: https://www.youtube.com/watch?v=5Ay5GqJwHF8
[youtube] 5Ay5GqJwHF8: Downloading webpage
[youtube] 5Ay5GqJwHF8: Downloading ios player API JSON
[youtube] 5Ay5GqJwHF8: Downloading android player API JSON
[youtube] 5Ay5GqJwHF8: Downloading m3u8 information
[info] 5Ay5GqJwHF8: Downloading 1 format(s): 140
[download] /Users/bpakra200/Downloads/YouTubeclip/If You Build It, He Will Come - Field of Dreams (1⧸9) Movie CLIP (1989) HD.m4a has already been downloaded
[download] 100% of    2.28MiB
[ExtractAudio] Not converting audio /Users/bpakra200/Downloads/YouTubeclip/If You Build It, He Will Come - Field of Dreams (1⧸9) Movie CLIP (1989) HD.m4a; file is already in target format m4a
Transcribing part 1!
After load
page_content="♪♪ ♪♪ ♪♪ If you build it, he will come. ♪♪ ♪♪ I

In [21]:
# Returns a list of Documents, which can be easily viewed or parsed
len(docs)

1

In [22]:
# Combine doc
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)

Although HuggingFaceEmbeddings is a free and user-friendly option to get started with embeddings, OpenAI also offers an API for text embeddings5, which might perform much better. With LangChain, it is straightforward to use the OpenAI API for embeddings:

# 2 Create Embedding

In [23]:
embeddings = OpenAIEmbeddings()

# Storing Embeddings
Note that Chroma doesn't automatically persist the embeddings, so you'll need to provide an additional parameter persist_directory to ensure they're saved along with the documents:

In [8]:
PERSIST_DIRECTORY_OPENAI = "../tubegptdbclip"
 
tubegptdbclip = Chroma.from_documents(
    documents=docs, embedding= embeddings, persist_directory=PERSIST_DIRECTORY_OPENAI
)
tubegptdbclip.similarity_search_with_score("Which moive?", k=2)

[(Document(page_content="♪♪ ♪♪ ♪♪ If you build it, he will come. ♪♪ ♪♪ If you build it, he will come. ♪♪ Hey, Annie! Annie, what was that? What was what? That voice just now, what was it? We didn't hear anything. All right. If you build it, he will come. Okay, you must have heard that. Sorry. Come on in for dinner. Let's go, Parker. ♪♪ ♪♪ ♪♪ ♪♪", metadata={'source': '/Users/bpakra200/Downloads/YouTubeclip/If You Build It, He Will Come - Field of Dreams (1⧸9) Movie CLIP (1989) HD.m4a', 'chunk': 0}),
  0.493926465511322),
 (Document(page_content="♪♪ ♪♪ ♪♪ If you build it, he will come. ♪♪ ♪♪ If you build it, he will come. ♪♪ Hey, Annie! Annie, what was that? What was what? That voice just now, what was it? We didn't hear anything. All right. If you build it, he will come. Okay, you must have heard that. Sorry. Come on in for dinner. Let's go, Parker. ♪♪ ♪♪ ♪♪ ♪♪", metadata={'source': '/Users/bpakra200/Downloads/YouTubeclip/If You Build It, He Will Come - Field of Dreams (1⧸9) Movie CLIP 

# Loading Embeddings
Loading the embeddings is just as easy, you just need to provide the persist_directory and embedding_function parameters:

# save openai embedding

In [9]:
tubegptdbclip.persist()

In [57]:
PERSIST_DIRECTORY_OPENAI = "../tubegptdbclip"
embeddings = OpenAIEmbeddings()
tubegptdbclip = Chroma(persist_directory=PERSIST_DIRECTORY_OPENAI, embedding_function=embeddings)
tubegptdbclip.similarity_search_with_score("Which moive?", k=2)

[(Document(page_content="♪♪ ♪♪ ♪♪ If you build it, he will come. ♪♪ ♪♪ If you build it, he will come. ♪♪ Hey, Annie! Annie, what was that? What was what? That voice just now, what was it? We didn't hear anything. All right. If you build it, he will come. Okay, you must have heard that. Sorry. Come on in for dinner. Let's go, Parker. ♪♪ ♪♪ ♪♪ ♪♪", metadata={'source': '/Users/bpakra200/Downloads/YouTubeclip/If You Build It, He Will Come - Field of Dreams (1⧸9) Movie CLIP (1989) HD.m4a', 'chunk': 0}),
  0.493926465511322),
 (Document(page_content="♪♪ ♪♪ ♪♪ If you build it, he will come. ♪♪ ♪♪ If you build it, he will come. ♪♪ Hey, Annie! Annie, what was that? What was what? That voice just now, what was it? We didn't hear anything. All right. If you build it, he will come. Okay, you must have heard that. Sorry. Come on in for dinner. Let's go, Parker. ♪♪ ♪♪ ♪♪ ♪♪", metadata={'source': '/Users/bpakra200/Downloads/YouTubeclip/If You Build It, He Will Come - Field of Dreams (1⧸9) Movie CLIP 

# create Audio processor for Youtube Video
##  Prompt Engineering Template

In [58]:
template = """You're a Professor who helps students on Youtube contents .
 
{context}
 
Answer with Accurate data   to the question and the way Professor speaks and only depends on provided Data. 
 
Question: {question}
Answer:"""
 
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
print(
    prompt.format(
        context="A youtube user asking about contents ",
        question="Why do we need to zero out the gradient before backprop at each step?",
    )
)

You're a Professor who helps students on Youtube contents .
 
A youtube user asking about contents 
 
Answer with Accurate data   to the question and the way Professor speaks and only depends on provided Data. 
 
Question: Why do we need to zero out the gradient before backprop at each step?
Answer:


## implement Retriver and Reader with Vector database for Audio input

In [80]:
chain_type_kwargs = {"prompt": prompt}
audio_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=tubegptdbclip.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
    
    
)

# Test few questions on audio

In [81]:
query = "what was the phase the person hearing again and again?"
response = audio_chain.run(query)
print_response(response)

The person was hearing the phrase "If you build it, he will come" repeatedly.


In [82]:
query = "Why the perosn ask a lady to reconfirm the hearing"
response = audio_chain.run(query)
print_response(response)

The person asks the lady to reconfirm the hearing because they heard a voice that the lady claims
not to have heard. They want to make sure that they both heard the same thing and that it wasn't
just their imagination.


##  Vision process  After Captioning . 
### Captions need GPU which will be in Separate Notebook in colab with high end GPU
### GPU based notebook ceate a caption Text files with captioning images in camera shot in colab 


In [100]:
loader = TextLoader('/Users/bpakra200/tubegpt/tubegpt/src/notebook/list-blip2-moiveClip.txt')

docs_vision_caption = loader.load()

In [101]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
splits = text_splitter.split_documents(docs_vision_caption)

In [97]:
def create_prompt_details_decribtion():
  

    template = """you are a Video content describer   and create a details describtion of a video depends on    
     captions of all images   . All captions are in sequence of the video 
    {image_captions}
    Create a Title :
    Create  a Full Descrption :
    """
    prompt = PromptTemplate(input_variables=["image_captions"], template=template)
    return prompt

In [102]:
 video_descrption_prompt = create_prompt_details_decribtion()
 llm = ChatOpenAI(temperature=0)
 video_descrption_chain = LLMChain(
        llm=llm, prompt=video_descrption_prompt, verbose=True,)

In [103]:


processed_splits = []
for split in splits:
    result = video_descrption_chain({"image_captions": split}, return_only_outputs=True)
    processed_splits.append(result['text'])



> Entering new LLMChain chain...
Prompt after formatting:
you are a Video content describer   and create a details describtion of a video depends on    
     captions of all images   . All captions are in sequence of the video 
    page_content='an image of a cloudy sky with light in the middle\na picture of a light that is inside a cloud\na single lightning bolt glows in the dark sky\nan image of some clouds in the sky\na cloud with a moon in the dark sky\na view of clouds that appear to be falling from above\nsun rays in the distance over a field\nharry potter and the sword in the stone / lightning / water / land / horizon\na thunderstorm is over the field with trees\nsun rays light up over a field\nrainbow on cloudy background and landscape\nsunlight through rain clouds over a field with trees\nsun rising behind a purple cloud during a storm\na picture of a field in the middle of a dark sky\nraindrops against a blue sky\nthe sun shining through clouds from the horizon in the sky\n

## process image caption to Full description of video

In [11]:


with open("clip-descrption-blip2v2.txt", "w") as file:
    for item in processed_splits:
        file.write(str(item) + "\n")

## Saving Description in Vector Database

In [13]:
loader = TextLoader('/Users/bpakra200/tubegpt/tubegpt/src/notebook/clip-descrption-blip2v2.txt')

docs_vision_description = loader.load()

In [ ]:
text_splitter_vision_description = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts_vision_description = text_splitter_vision_description.split_documents(docs_vision_description)
len(texts_vision_description)

In [25]:
embeddings = OpenAIEmbeddings()

In [ ]:
PERSIST_DIRECTORY_OPENAI = "../tubegptvisionclipblip2db/"
 
tubegptvisionclipblip2db = Chroma.from_documents(
    documents=texts_vision_description, embedding= embeddings, persist_directory=PERSIST_DIRECTORY_OPENAI
)
tubegptvisionclipblip2db.similarity_search_with_score("Who is the person with white shirt?", k=2)

In [19]:

tubegptvisionclipblip2db.persist()

## load Vectors from database

In [27]:
PERSIST_DIRECTORY_OPENAI =  "../tubegptvisionclipblip2db"
embeddings = OpenAIEmbeddings()
tubegptvisionclipblip2db = Chroma(persist_directory=PERSIST_DIRECTORY_OPENAI, embedding_function=embeddings)
tubegptvisionclipblip2db.similarity_search_with_score("Who is the person with white shirt?", k=2)

[(Document(page_content="Full Description: In this captivating video, we witness a man's exploration of a vast corn field. The video begins with a shot of the man standing amidst the towering cornstalks, his gaze fixed upon the sky above. As the camera pans, we get a closer look at the man, dressed in a crisp white shirt, as he becomes the focal point of the frame. This iconic shot, reminiscent of a movie scene, features Mark Ruffalo and his wife Gisele, adding an element of intrigue to the narrative.\n\nThe next sequence showcases a man in a blue shirt, positioned to the left of the frame. He appears to be part of a movie clip, adding an element of mystery to the video. The scene transitions to another man, this time holding a baseball bat, standing confidently in the corn field. The juxtaposition of the man's presence with the peaceful surroundings creates a sense of anticipation.", metadata={'source': '/Users/bpakra200/tubegpt/tubegpt/src/notebook/clip-descrption-blip2v2.txt'}),
  0

## Create a prompt Templete

In [83]:
template = """You're a Helper of blind person who needs help to understand a visual of a video .He can understand audio part of the video  .
 
{context}
 
Answer with Accurate data   to the question and the way Helper of  poor vision person speaks and only depends on provided Data. 
 
Question: {question}
Answer:"""
 
vistion_prompt = PromptTemplate(template=template, input_variables=["context", "question"])
print(
    prompt.format(
        context="A blind person asking about visual Content ",
        question="Who is the person with white shirt?",
    )
)

You're a Professor who helps students on Youtube contents .
 
A blind person asking about visual Content 
 
Answer with Accurate data   to the question and the way Professor speaks and only depends on provided Data. 
 
Question: Who is the person with white shirt?
Answer:


# Create Vision chain 

In [84]:
chain_type_kwargs = {"prompt": vistion_prompt}
vision_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=.4),
    chain_type="stuff",
    retriever=tubegptvisionclipblip2db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)

In [71]:
query = "Who is the person with white shirt? "
response = vision_chain.run(query)
print_response(response)

The person with the white shirt is Mark Ruffalo.


In [31]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

## Create Templarte for Agent

In [85]:
audio_template = """You are a very smart video content descriptor for deaf people. \
You are great at answering questions about Audio part of video in a concise and easy to understand manner. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


vision_template = """You are a very smart video content descriptor for blind people.  You are great at answering  questions on visual part of video
Content. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [48]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

## Create Tool for Agent

In [87]:
tools = [
    Tool(
        name="A video deacriptor for deaf people",
        func=audio_chain.run,
        description=audio_template,
    ),
    Tool(
        name="A video deacriptor for Blind people",
        func=vision_chain.run,
        description=vision_template,
    ),
]

In [88]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

## Asking qustion to agent . Agent is using REACT framework and chain of thoughts

In [89]:
agent.run(
    "Who is the  actor  with white shirt?"
)



> Entering new AgentExecutor chain...
 I need to look for information about the actor wearing a white shirt.
Action: A video deacriptor for Blind people
Action Input: Look for visual information about the actor with a white shirt.
Observation: In the video, there is an actor wearing a crisp white shirt. He is standing amidst the towering cornstalks and his gaze is fixed upon the sky above. He becomes the focal point of the frame, resembling a movie scene. This actor is Mark Ruffalo, who is accompanied by his wife Gisele.
Thought: I now know the final answer
Final Answer: The actor with the white shirt is Mark Ruffalo.

> Finished chain.


'The actor with the white shirt is Mark Ruffalo.'

In [91]:
agent.run(
    "what was color of the shirt ?"
)



> Entering new AgentExecutor chain...
 What information can I use to answer this question?
Action: A video deacriptor for Blind people
Action Input: Observing the video
Observation: Based on the provided data, the video starts with a group of people sitting on a porch of an old house in a peaceful countryside setting. Then, it shows a woman sitting outside the house in the evening, possibly reflecting on memories or enjoying the tranquility. The scene transitions to a cornfield, where a person is seen walking through the tall stalks, creating a sense of solitude and contemplation. There is also a young girl exploring the cornfield. The video then shifts to a woman and her daughter sitting on a swing on a porch, capturing a joyful and innocent moment.
Thought: I will need to observe what color of shirt the person is wearing. 
Action: A video deacriptor for Blind people
Action Input: Observing the video
Observation: Based on the provided data, the video begins by showing a group of peo

'The color of the shirt was blue.'

In [92]:
agent.run(
    "Why the perosn ask a lady to reconfirm the hearing?"
)





> Entering new AgentExecutor chain...
 The person might be asking if the lady heard correctly, or they might be asking her to repeat what they said. 
Action: A video deacriptor for deaf people
Action Input: Analyze the audio part of the video 
Observation: Based on the provided data, the audio part of the video consists of a conversation between two individuals, Annie and the Professor. Annie seems to be unaware of a voice that the Professor heard, suggesting that it may have been a hallucination or something only the Professor could hear. The Professor mentions the phrase "If you build it, he will come" multiple times, indicating that it is significant in some way. The conversation ends with Annie inviting the Professor and Parker for dinner.
Thought: This conversation may be related to why the Professor asked the lady to reconfirm the hearing. 
Action: A video deacriptor for Blind people
Action Input: Analyze the visual part of the video
Observation: The visual part of the video st

'The Professor was likely asking the lady to reconfirm her hearing because of the peaceful and nostalgic visuals presented in the video, which could be connected to the phrase "If you build it, he will come" and the invitation to dinner.'

In [93]:
agent.run(
    "who is saying If you build it, he will come?"
)



> Entering new AgentExecutor chain...
 I need to find out who is speaking this line
Action: A video deacriptor for Blind people
Action Input: Describe the visuals of the scene
Observation: The visuals of the scene include sun rays piercing through the distance, casting a warm glow over a picturesque field. There is a reference to the tale of "Harry Potter and the Sword in the Stone," symbolizing the convergence of elements such as lightning, water, land, and the horizon. A thunderstorm scene unfolds, engulfing a field with trees, showcasing the power of nature. The video then transitions to a serene moment with sun rays illuminating a field, creating a sense of tranquility and hope. A stunning rainbow against a cloudy backdrop and landscape captures the viewer's attention. Sunlight pierces through rain clouds, casting a magical glow over a field with trees. The video also showcases the sun rising behind a purple cloud during a storm, creating a dramatic and awe-inspiring visual.
Thou

'The line "If you build it, he will come" is spoken in the video by an unknown narrator.'